### Import Libraries

In [1]:
!pip3 install -e /mnt/lab/users/maxgagnon/src/s2p-lbm/suite2p

Obtaining file:///mnt/lab/users/maxgagnon/src/s2p-lbm/suite2p
  Preparing metadata (setup.py) ... done
  Attempting uninstall: suite2p
    Found existing installation: suite2p 0.10.4.dev65+gabd1f66.d20240305
    Uninstalling suite2p-0.10.4.dev65+gabd1f66.d20240305:
      Successfully uninstalled suite2p-0.10.4.dev65+gabd1f66.d20240305
  Running setup.py develop for suite2p

[notice] A new release of pip is available: 23.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
plt.style.use('seaborn')
import time
import napari

## !! CHANGE ME !! ##
from suite2p.io import lbm as lbmio
from suite2p.suite3d import utils, init_pass
from suite2p.suite3d.job import Job
from suite2p.suite3d import ui
import suite2p

os.chdir('/mnt/lab/users/maxgagnon/src/s2p-lbm')
from utils import tiff_utils as tfu
import numpy as np
from skimage.io import imread

/tmp/ipykernel_30534/4212160794.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


# 0. Set initial parameters

In [2]:
# ~0 seconds

## Find all the tiff files in the tif path

# tif_path = r'/mnt/lab/users/maxgagnon/cache/LBM/ExampleData_Ali_2023_09_13'
# tif_path = r'/mnt/lab/users/maxgagnon/cache/LBM/ExampleData_Ali/sample LBM data from s2p-lbm repo'
tif_path = r'/mnt/scratch11/vaziri_lbm_demo_scan/'
tifs = tfu.get_tif_paths(tif_path)
tifs = [tifs[30]]
tifs

['/mnt/scratch11/vaziri_lbm_demo_scan/29513-1-8_00001_00031.tif']

In [4]:
# Set the mandatory parameters
params = {
    # volume rate
    'fs': tfu.get_vol_rate(tifs[0]),
    # planes to analyze. 0 is deepest, 30 is shallowest (corrected for ScanImage channel IDs)
    'planes' : n.arange(30),
    # Decay time of the Ca indicator in seconds. 1.3 for GCaMP6s. This example is for GCamP8m
    'tau' : 1.3}


params

{'fs': 5.93921,
 'planes': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 'tau': 1.3}

In [5]:
# Create the job
job = Job('/mnt/lab/users/maxgagnon/cache/LBM/OurDataTesting/', 
          tifs = tifs,
          params = params, 
          create = True, 
          overwrite = True, 
          verbosity = 3,
          job_id = 'test_1')


# optional parameters for initialization
# load 1 file to initialize, and select 200 frames randomly to do initalization on
job.params['n_init_files']  = 1
job.params['init_n_frames'] = 200


# Set to None to auto-compute the crosstalk coefficient
# You can set a float value between 0-1 (usually around 0.1-0.3) to override the calculation
job.params['override_crosstalk'] = .2

Job directory /mnt/lab/users/maxgagnon/cache/LBM/OurDataTesting/s3d-test_1 already exists
Loading job directory for test_1 in /mnt/lab/users/maxgagnon/cache/LBM/OurDataTesting/
   Loading dirs 
      Found dir registered_data
      Found dir summary
      Found dir iters
   Loading default params
      Updating param fs
      Updating param planes
      Updating param tau
   Updated main params file


# 0. Print metadata about tiffs

In [ ]:
job.run_init_pass()

   Saved a copy of params at /mnt/lab/users/maxgagnon/cache/LBM/OurDataTesting/s3d-test_1/summary
   Updated main params file
Launching initial pass
Saving summary to /mnt/lab/users/maxgagnon/cache/LBM/OurDataTesting/s3d-test_1/summary/summary.npy
Loading /mnt/scratch11/vaziri_lbm_demo_scan/29513-1-8_00001_00031.tif


<tifffile.TiffFile '29513-1-8_00001_00031.tif'> asarray failed to reshape (24120, 4024, 146) to (50000, 30, 4024, 146)


In [ ]:
init_tifs = init_pass.choose_init_tifs(tifs, 
                                       job.params['n_init_files'], 
                                       job.params['init_file_pool'], 
                                       job.params['init_file_sample_method'])

n_ch_tif = job.params.get('n_ch_tif', 30)

init_mov = init_pass.load_init_tifs(init_tifs, 
                                    job.params['planes'], 
                                    job.params['notch_filt'], 
                                    n_ch_tif = n_ch_tif,
                                    convert_plane_ids_to_channel_ids = job.params.get('convert_plane_ids_to_channel_ids', True))
                                    

Loading /mnt/scratch11/vaziri_lbm_demo_scan/29513-1-8_00001_00031.tif


<tifffile.TiffFile '29513-1-8_00001_00031.tif'> asarray failed to reshape (24120, 4024, 146) to (50000, 30, 4024, 146)


In [ ]:
nz, nt, ny, nx = init_mov.shape
print(f'nt size: {nt}')
print(f"init_n_frames size: {job.params['init_n_frames']}")

print(init_mov.shape)

# 1. Initialization

In [ ]:
# ~33 Minutes
job.run_init_pass()

### How'd Registration go?

In [ ]:
job.show_summary_plots()

In [ ]:
# summary_dir = '/mnt/lab/users/maxgagnon/cache/LBM/ExampleData_Ali_2023_09_13/s3d-test_1/summary'
summary_dir = '/mnt/lab/users/maxgagnon/cache/LBM/OurDataTesting/s3d-test_1/summary'
summary = n.load(summary_dir + '/summary.npy', allow_pickle=True)
summary = summary.item()

In [ ]:
print(summary.keys())
print(f"----------\ncrosstalk_coeff: {summary['crosstalk_coeff']}")
print(f"----------\nplane_shifts: \n{summary['plane_shifts']}")
print(f"----------\nmin_pix_vals: \n{summary['min_pix_vals']}")
# print(summary['plane_shifts'])

Raw Images

In [ ]:
fig, ax = plt.subplots(5,6, figsize=(20,20))
ax = ax.flatten()
for idx, a in enumerate(ax):
    a.imshow(summary['raw_img'][idx,:,:], 'viridis')
    a.set_yticks([])
    a.set_xticks([])

Images

In [ ]:
fig, ax = plt.subplots(5,6, figsize=(20,20))
ax = ax.flatten()
for idx, a in enumerate(ax):
    a.imshow(summary['img'][idx,:,:], 'viridis')
    a.set_yticks([])
    a.set_xticks([])

Unaligned Reference Images

In [ ]:
fig, ax = plt.subplots(5,6, figsize=(20,20))
ax = ax.flatten()
for idx, a in enumerate(ax):
    a.imshow(summary['ref_img_3d_unaligned'][idx,:,:], 'viridis')
    a.set_yticks([])
    a.set_xticks([])

Aligned Reference Images

In [ ]:
fig, ax = plt.subplots(5,6, figsize=(20,20))
ax = ax.flatten()
for idx, a in enumerate(ax):
    a.imshow(summary['ref_img_3d'][idx,:,:], 'viridis')
    a.set_yticks([])
    a.set_xticks([])

# 2. Registration and Fusion
- Suite2p Registration
- Speedup: Pipeline file I/O and computation (you don't have to transfer all your files to local storage)
- Accounts for plane shifts
- Fuses the strips

Parameters

In [ ]:
# split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

# FUSING: fuse the strips from the FOV into one image
# this is the number of pixels to skip between pixels, 13 makes it look relatively smooth
job.params['n_skip'] = 13

# should only fuse files where the plane_shifts look good, otherwise you might get an error
job.params['fuse_crop'] = ((0,22), (0,584), (0,584))

Registration

In [ ]:
# ~6 minutes
job.register()

Fuse the registered sections

In [ ]:
# ~2 minutes
job.fuse_registered_movie(n_proc=1)

In [ ]:
# ~40 seconds
mov_full = job.get_registered_movie('registered_fused_data', 'fused')
im_full = mov_full[:,:1000].mean(axis=1).compute()

In [ ]:
mov_full[:,:1000]

In [ ]:
print(mov_full.shape)
print(im_full.shape)

Figures

In [ ]:
fig, ax = plt.subplots(5,6, figsize=(15,15))
ax = ax.flatten()
for idx, a in enumerate(ax):
    a.imshow(im_full[idx], 'viridis')
    a.set_yticks([])
    a.set_xticks([])

# 3. SVD Denoising

Parameters

In [ ]:
job.params['n_svd_comp'] = 500
job.params['svd_crop'] = ((0,22), (55,584), (0, 460))
job.params['svd_block_shape'] = (4,200,200)
job.params['svd_block_overlaps'] = (2,100,100)
svd_dir_tag = 'svd_cropped'

svd

In [ ]:
job.make_new_dir(svd_dir_tag)
svd_info = job.svd_decompose_movie(svd_dir_tag, run_svd=True)

In [ ]:
# if 'job.dirs[svd_dir_tag]' in locals() and os.path.exists(job.dirs[svd_dir_tag] + '/svd_info.npy'):
if 'svd_dir_tag' in job.dirs:
    print(f"Existing file found: {os.path.join(job.dirs[svd_dir_tag], 'svd_info.npy')}")
    svd_info = n.load(os.path.join(job.dirs[svd_dir_tag], 'svd_info.npy'),allow_pickle=True).item()
else:
    # ~ 1 hour
    print('Existing file NOT found; Creating new svd_info. This process takes ~ 1 hour')
    job.make_new_dir(svd_dir_tag)
    svd_info = job.svd_decompose_movie(svd_dir_tag, run_svd=True)

In [ ]:
print(svd_info.keys())
print(f"n_blocks: {svd_info['n_blocks']}")
print(f"block_shape: {svd_info['block_shape']}")
print(f"block_overlaps: {svd_info['block_overlaps']}")
# print(f"blocks: {svd_info['blocks']}")
print(f"grid_shape: {svd_info['grid_shape']}")
print(f"mov_shape: {svd_info['mov_shape']}")
print(f"n_comps: {svd_info['n_comps']}")
print(f"svd_dirs: {svd_info['svd_dirs']}")

# 4. Correlation Map (Sweep through correlation map parameters)
Steps:
1. Temporal High-Pass Filter
2. Normalize Pixel intensity
3. Remove neuropil & highlight cells w/ spatial filters
4. Threshold filtered activity to include only calcium events
5. Accumulate over time to calculate correlation map

In [ ]:
# Parameters
job.params['t_batch_size'] = 400
job.params['n_proc_corr'] = 8
job.params['n_svd_comps'] = 200
job.params['intensity_thresh'] = 0.25

params_to_sweep = {
                    'intensity_thresh' : (0.25, 0.4),
                    'n_svd_comp' : (200,300,400),
                }

In [ ]:
# if 'job.dirs[sweep-01]' in locals() and os.path.exists(job.dirs['sweep-01']+ '/sweep_summary.npy'):
if  os.path.exists(job.dirs['sweep-01']+ '/sweep_summary.npy'):
    print(f"Existing file found: {os.path.join(job.dirs['sweep-01'], 'sweep_summary.npy')}")
    sweep_summary = n.load(os.path.join(job.dirs['sweep-01'], 'sweep_summary.npy'),allow_pickle=True).item()
else:
    # ~ 8 minutes
    print('Existing file NOT found; Creating new sweep_summary. This process takes ~ 10 Minutes')
    sweep_summary = job.sweep_params(params_to_sweep, 
                                    svd_info,
                                    testing_dir_tag = 'sweep-01',
                                    do_vmap = True, 
                                    all_combinations = True, 
                                    n_test_iters = 1)

In [ ]:
sweep_summary = job.sweep_params(params_to_sweep, 
                                svd_info,
                                testing_dir_tag = 'sweep-01',
                                do_vmap = True, 
                                all_combinations = True, 
                                n_test_iters = 1)

In [ ]:
# PLOT: a slice of the image stack for each of the (4) vmaps param options. What should I be looking for here?
print(np.asarray(sweep_summary['vmaps']).shape)
fig, ax = plt.subplots(2,2,figsize=(10,10))
ax = ax.flatten()
for idx, a in enumerate(ax):
    a.imshow(np.asarray(sweep_summary['vmaps'])[idx,2,:,:], 'viridis')

In [ ]:
job.params['intensity_thresh'] = 0.4
job.params['n_svd_comps'] = 300
job.params['t_batch_size'] = 300

In [ ]:
vmap, mean_img, max_img = job.calculate_corr_map(mov = svd_info)

# 5. Detect, Extract & Visualize masks
- Choose a threshold in the correlation map
- Generally robust and fast! Easy to test out on small samples (can implement easy itnerface for parameter sweep)
- Uses 'mov_sub' - temporary filtered movie from corr-map calculation

In [ ]:
sweep_summary = n.load(os.path.join(job.dirs['sweep-01'], 'sweep_summary.npy'),allow_pickle=True).item()

In [ ]:
job.params['detection_timebin'] = 1 
job.params['detection_time_crop'] = (None,None)
job.params['max_pix'] = 250
job.params['peak_thresh'] = 5.8
job.params['percentile'] = 99.0
job.params['max_iter'] = 10000 # maximum number of ROIs detected in given patch

In [ ]:
# ~ 21 minutes
combined_dir = job.patch_and_detect(combined_name='combined-full')

### Deconvolution & Traces

In [ ]:
# ~ 22 seconds
traces = job.extract_and_deconvolve(stats_dir = combined_dir)
traces

In [ ]:
np.random.choice(np.arange(traces['F'].shape[0]),1)

In [ ]:
traces['F'].shape

In [ ]:
# Plot Traces
traces['F'].shape
fig, ax = plt.subplots(3,1,figsize=(20,20))
ax = ax.flatten()

all = 1
neuron2plot = np.random.choice(np.arange(traces['F'].shape[0]),1)[0]

if all:
    f = ax[0].plot(traces['F'])
    f = ax[1].plot(traces['Fneu'])
    f = ax[2].plot(traces['spks'])
else:
    f = ax[0].plot(traces['F'][neuron2plot,:])
    f = ax[1].plot(traces['Fneu'][neuron2plot,:])
    f = ax[2].plot(traces['spks'][neuron2plot,:])

    
ax[0].set_title('F')
# ax[0].set_ylim([-10, 500])
ax[1].set_title('Fneu')
# ax[1].set_ylim([-10, 250])
ax[2].set_title('spks')
# ax[2].set_ylim([-10, 200])

print(f"Neuron: {neuron2plot}")